In [182]:
import numpy as np

class BasicNNModule:
    def __init__(self):
        self.lr = 0.001

class SingleLinear(BasicNNModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weights = np.ones((out_features, in_features)) * 0.5
        self.bias = np.zeros(out_features)

    def forward(self, x):
        return np.matmul(x, self.weights.T) + self.bias
    
    def backward(self, delta_next_layer, w_next_layer, b_next_layer, a_cur_layer, y_last_layer):
        # print(d2y.shape, y2i.shape, y_last_layer.shape)
        # delta = d2y * y2i
        delta = (np.matmul(delta_next_layer.T, w_next_layer) * a_cur_layer).T
        # print(delta)
        # L2 regularization
        self.weights += self.lr * (np.matmul(delta, y_last_layer) - 0.01 * self.weights)
        # self.weights += self.lr * np.matmul(delta, y_last_layer)
        self.bias += self.lr * delta.T[0]
        return delta, self.weights, self.bias
    
    def softmax_backward(self, y, label, y_last_layer):
        delta = (y - label).T
        # print('before update:', self.weights, self.bias)
        # print(delta.shape, y_last_layer.shape, self.bias.shape)

        # L2 regularization
        self.weights += self.lr * (np.matmul(delta, y_last_layer) - 0.01 * self.weights)
        # self.weights += self.lr * np.matmul(delta, y_last_layer)

        self.bias += self.lr * delta.T[0]
        # print('after update:', self.weights, self.bias)
        return delta, self.weights, self.bias

class ReLU:
    def forward(self, x):
        return np.maximum(x, 0)
    
    def backward(self, x):
        return np.where(x > 0, 1, 0)
    
class Softmax:
    def forward(self, x):
        # 減去最大值以防止 overflow
        x = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    


In [ ]:
import numpy as np

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))
inputs = inputs[idx]
labels = labels[idx]


linear_1 = SingleLinear(4, 5)
relu_1 = ReLU()
linear_2 = SingleLinear(5, 5)
relu_2 = ReLU()
linear_3 = SingleLinear(5, 3)
softmax_3 = Softmax()

for _ in range(5):

    for input, label in zip(inputs, labels):
        input = input.reshape(1, -1)
        # print(input, label)
        i_1 = linear_1.forward(input)
        y_1 = relu_1.forward(i_1)
        i_2 = linear_2.forward(y_1)
        y_2 = relu_2.forward(i_2)
        i_3 = linear_3.forward(y_2)
        print("i_3", i_3)
        y_3 = softmax_3.forward(i_3)
        print("y_3", y_3)
        print("label", label)
        delta_3, w_3, b_3 = linear_3.softmax_backward(y_3, label, y_2)
        delta_2, w_2, b_2 = linear_2.backward(delta_3, w_3, b_3, relu_2.backward(i_2), y_1)
        delta_1, w_1, b_1 = linear_1.backward(delta_2, w_2, b_2, relu_1.backward(i_1), input)
        # print('delta_3:\n', delta_3)
        # print('w_3:\n', w_3)
        # print('b_3:\n', b_3)
        
        # print('delta_2:\n', delta_2)
        # print('w_2:\n', w_2)
        # print('b_2:\n', b_2)
        # break
        # delta_1 = linear_1.backward(- ())
    

In [135]:
np.matmul(delta_3.T, w_3)

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [171]:
class Softmax:
    def forward(self, x):
        # 減去最大值以防止 overflow
        x = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
Softmax().forward(np.array([[1, 5, 2]]))

array([[0.01714783, 0.93623955, 0.04661262]])

In [236]:
a = np.random.randn(3, 4)
b = np.random.randn(4)
print(a)
print(b)
print(a.dot(b))

[[-0.95799297  0.50648367 -1.29696496  0.99630143]
 [-0.48203369  0.28668749  2.12959588  0.65556684]
 [-0.08816886  0.089359   -0.41808245  0.38364675]]
[ 0.09246073  0.36246263  0.09009849 -0.19246959]
[-0.21360764  0.12504099 -0.08727179]


In [264]:
import numpy as np

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))
inputs = inputs[idx].T
labels = labels[idx].T


inputs.shape, labels.shape

((4, 150), (3, 150))

In [386]:
m, n = inputs.T.shape

print('m:', m, 'n:', n)

def init_params():
    W1 = np.random.rand(10, 4) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(3, 10) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    # print('X shape:', X.shape, 'W1 shape:', W1.shape)
    Z1 = np.matmul(W1, X) + b1
    A1 = ReLU(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, A2, W2, X, Y):
    one_hot_Y = one_hot(Y)
    print('A2 shape:', A2.shape, 'Y shape:', Y.shape, 'one_hot_Y shape:', one_hot_Y.shape)
    dZ2 = A2 - one_hot_Y
    # dW2 = 1 / m * dZ2.dot(A1.T)
    # print('dZ2 shape:', dZ2.shape, 'A1 shape:', A1.shape)
    dW2 = (1 / m) * np.matmul(dZ2, A1.T)
    # print(dW2.shape)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    # dW1 = 1 / m * dZ1.dot(X.T)
    dW1 = (1 / m) * np.matmul(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        # for X, Y in zip(X_all.T, Y_all.T):
        #     X = X.reshape(-1, 1)
        #     Y = Y.reshape(-1, 1)
            # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
            # print('X_all shape:', X_all.shape)
            # print('X shape:', X.shape)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)

            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
        break
    return W1, b1, W2, b2

m: 150 n: 4


In [387]:
X_train = inputs
Y_train = np.argmax(labels, axis=0)
# Y_train = labels
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

A2 shape: (3, 150) Y shape: (150,) one_hot_Y shape: (3, 150)
Iteration:  0
0.31333333333333335


In [473]:
m, n = inputs.T.shape

print('m:', m, 'n:', n)

def init_params():
    W1 = np.random.rand(10, 4) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(3, 10) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    # print('X shape:', X.shape, 'W1 shape:', W1.shape)
    Z1 = np.matmul(W1, X) + b1
    A1 = ReLU(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, A2, W2, X, Y):
    # one_hot_Y = Y.reshape(-1, 1)
    one_hot_Y = one_hot(Y)
    # print('A2 shape:', A2.shape, 'Y shape:', Y.shape, 'one_hot_Y shape:', one_hot_Y.shape)
    dZ2 = A2 - one_hot_Y
    # dW2 = 1 / m * dZ2.dot(A1.T)
    # print('dZ2 shape:', dZ2.shape, 'A1 shape:', A1.shape)
    dW2 = (1 / m) * np.matmul(dZ2, A1.T)
    # print(dW2.shape)
    db2 = (1 / m) * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    # dW1 = 1 / m * dZ1.dot(X.T)
    dW1 = (1 / m) * np.matmul(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    bs = 10
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        # for X, Y_s in zip(X_all.T, Y_all.T):

        #     X = X.reshape(-1, 1)
        #     Y = Y.reshape(-1, 1)
        # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
        # print('X_all shape:', X_all.shape)
        # print('X shape:', X.shape)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)

            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
        # break
    return W1, b1, W2, b2

m: 150 n: 4


In [476]:
X_train = inputs
Y_train = np.argmax(labels, axis=0)
# Y_train = labels
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.05, 101)

Iteration:  0
0.3333333333333333
Iteration:  50
0.9133333333333333
Iteration:  100
0.9666666666666667


# =======================================

In [732]:
def init_params():
    # W1 = np.random.rand(4, 10) - 0.5  # Changed from (10, 4) to (4, 10)
    # b1 = np.random.rand(1, 10) - 0.5  # Changed from (10, 1) to (1, 10)
    # W2 = np.random.rand(10, 3) - 0.5  # Changed from (3, 10) to (10, 3)
    # b2 = np.random.rand(1, 3) - 0.5   # Changed from (3, 1) to (1, 3)
    W1 = np.ones((4, 10))
    b1 = np.zeros((1, 10))
    W2 = np.ones((10, 3))
    b2 = np.zeros((1, 3))
    return W1, b1, W2, b2

def ReLU(x):
    return np.maximum(x, 0)

def d_ReLU(x):
    return x > 0

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    # X shape: (n_samples, n_features)
    I1 = np.matmul(X, W1) + b1  # Changed matrix multiplication order
    Y1 = ReLU(I1)
    I2 = np.matmul(Y1, W2) + b2  # Changed matrix multiplication order
    Y2 = softmax(I2)
    return I1, Y1, I2, Y2

def backward_prop(I1, Y1, Y2, W2, X, Y):

    m = X.shape[0]
    norm_factor = 1 / m
    
    dZ2 = Y2 - Y  # (n_samples, 3)
    # print('dZ2 shape:', dZ2.shape, 'A1.t shape:', A1.T.shape)
    dW2 = norm_factor * np.matmul(Y1.T, dZ2)  # Changed matrix multiplication order
    db2 = norm_factor * np.sum(dZ2, axis=0, keepdims=True)
    
    dZ1 = np.matmul(dZ2, W2.T) * d_ReLU(I1)  # Changed matrix multiplication order
    dW1 = norm_factor * np.matmul(X.T, dZ1)  # Changed matrix multiplication order
    db1 = norm_factor * np.sum(dZ1, axis=0, keepdims=True)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, axis=1)

def get_accuracy(predictions, Y):
    Y = np.argmax(Y, axis=1)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X_all, Y_all, X_val, Y_val, alpha, iterations, bs):
    # X_all -> (n_samples, n_features)
    # Y_all -> (n_samples, n_classes) one-hot 
    
    W1, b1, W2, b2 = init_params()
    
    X_batch = X_all.reshape(-1, bs, X_all.shape[1])
    Y_batch = Y_all.reshape(-1, bs, Y_all.shape[1])
    
    for i in range(iterations):
        loss = 0
        for X, Y in zip(X_batch, Y_batch):
            # X = X.reshape(1, -1)
            # Y = Y.reshape(1, -1)
            # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
            I1, Y1, I2, Y2 = forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = backward_prop(I1, Y1, Y2, W2, X, Y)
            W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
            loss += -np.sum(Y * np.log(Y2))

        if i % 1 == 0:
            print("Iteration: ", i)
            print("Loss: ", loss)
            _, _, _, pred = forward_prop(W1, b1, W2, b2, X_val)
            predictions = get_predictions(pred)
            print('Acc:', get_accuracy(predictions, Y_val))
            # print(predictions, Y)
        # break

    return W1, b1, W2, b2

In [733]:
import numpy as np

# 固定隨機種子，確保同參數下每次結果都相同
np.random.seed(1)

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))

X_data = inputs[idx]
# Y_train = np.argmax(labels, axis=0)

Y_data = labels[idx]

train_size = int(len(X_data) * 0.8)
val_size = len(X_data) - train_size

X_train, Y_train = X_data[:train_size], Y_data[:train_size]
X_val, Y_val = X_data[val_size:], Y_data[val_size:]

print(X_train.shape)

batch_size = 3
lr = 0.05

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, X_val, Y_val, lr, 30, batch_size)
# W1, b1, W2, b2

(120, 4)
Iteration:  0
Loss:  1001.7608705156697
Acc: 0.6583333333333333
Iteration:  1
Loss:  90.47267922908023
Acc: 0.6916666666666667
Iteration:  2
Loss:  75.40924592445778
Acc: 0.6916666666666667
Iteration:  3
Loss:  67.86493349199213
Acc: 0.7333333333333333
Iteration:  4
Loss:  62.505519460259904
Acc: 0.7583333333333333
Iteration:  5
Loss:  58.39690512425387
Acc: 0.775
Iteration:  6
Loss:  55.08818587889734
Acc: 0.8083333333333333
Iteration:  7
Loss:  52.32543400820918
Acc: 0.825
Iteration:  8
Loss:  49.95495163481795
Acc: 0.85
Iteration:  9
Loss:  47.878584861316476
Acc: 0.875
Iteration:  10
Loss:  46.030678749272724
Acc: 0.875
Iteration:  11
Loss:  44.36547332794938
Acc: 0.8916666666666667
Iteration:  12
Loss:  42.84989433520057
Acc: 0.8916666666666667
Iteration:  13
Loss:  41.459270293248174
Acc: 0.925
Iteration:  14
Loss:  40.17469989727743
Acc: 0.925
Iteration:  15
Loss:  38.98138057768009
Acc: 0.9416666666666667
Iteration:  16
Loss:  37.86751291851595
Acc: 0.9583333333333334


In [540]:
X_train.reshape(-1, 2, 4)

array([[[ 7.9566765e-01, -1.2495735e-01,  9.9022174e-01,  7.9059029e-01],
        [ 6.7449981e-01,  3.3784863e-01,  8.7649041e-01,  1.4479550e+00]],

       [[ 3.1099588e-01, -3.5636008e-01,  5.3529590e-01,  2.6469851e-01],
        [ 1.0380032e+00,  1.0644536e-01,  5.3529590e-01,  3.9617148e-01]],

       [[ 7.9566765e-01, -1.2495735e-01,  8.1962448e-01,  1.0535363e+00],
        [ 6.7449981e-01, -3.5636008e-01,  3.0783325e-01,  1.3322552e-01]],

       [[ 4.3216369e-01, -3.5636008e-01,  3.0783325e-01,  1.3322552e-01],
        [-2.9484364e-01, -3.5636008e-01, -9.0226904e-02,  1.3322552e-01]],

       [[ 5.5333203e-01, -1.2819721e+00,  6.4902753e-01,  3.9617148e-01],
        [-4.1601142e-01, -1.7447780e+00,  1.3723601e-01,  1.3322552e-01]],

       [[-5.3717923e-01, -1.2495735e-01,  4.2156458e-01,  3.9617148e-01],
        [-5.3717923e-01,  8.0065459e-01, -1.1706754e+00, -1.3129770e+00]],

       [[ 4.3216369e-01, -5.8776331e-01,  5.9216183e-01,  7.9059029e-01],
        [-1.5065227e+00,  